# Producing All Dataframes

In [4]:
apikey = "KNqUDtV7Kcktiuheo3EoNhB0zDlCevFAdqZrKgdj"
%pip install -r requirements.txt --upgrade
import fooddatacentral as fdc
import pandas as pd
import numpy as np
import re
from  scipy.optimize import linprog as lp
import warnings
warnings.simplefilter("ignore")

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


### **Dataframe Structures:**


| Dataset|Indexed by|Columns|
|---|---|---|
| Expenditures|i,t,m|j|
| Prices|j,u|t,m|
| Consumption|i,t,m,u|j|
| HH Characteristics|i,t,m|k|
| FCT|j,u|n|
| RDI|n|k|

where `i` indexes households, `t` indexes periods, `m` indexes
markets, `j` indexes goods, `k` indexes different kinds of household
characteristics, `u` indexes different unit names, and `n` indexes
different nutrients.  Finally, any RDI (&ldquo;recommended daily intake&rdquo;)
tables should be indexed by nutrients, with columns corresponding to
characteristics of persons within the household (e.g., age & sex
categories).

### **Uganda Data**

Expenditures

In [102]:
#Import csv file
ugd_x = pd.read_csv('Data Files/Uganda - Food Expenditures (2015-16).csv')
#Create pivot table with correct indexing
ugd_x = pd.pivot_table(ugd_x, values="Expenditure", index=["i","t","m"], columns="j" , aggfunc="median")
#Display
ugd_x.head()

,,j,Apples,Avocado,Beans (dry),Beans (fresh),Beef,Beer,Bread,Cabbage (green),Carrots,Cassava (dry),...,Sugar,Sweet Bananas,Sweet Potatoes (dry),Sweet Potatoes (flour),Sweet Potatoes (fresh),Tea,Tomatoes,Water,Watermelon,Wheat (flour)
i,t,m,,,,,,,,,,,,,,,,,,,,,
H0010101,2015-16,Central,NaN,500.0,3000.0,NaN,9000.0,NaN,2000.0,3000.0,NaN,NaN,...,5400.0,NaN,NaN,NaN,2000.0,200.0,2800.0,NaN,NaN,NaN
H0010201,2015-16,Central,NaN,NaN,2000.0,NaN,NaN,NaN,800.0,1500.0,NaN,NaN,...,2000.0,NaN,NaN,NaN,3000.0,150.0,600.0,NaN,NaN,NaN
H0010401,2015-16,Central,NaN,NaN,1800.0,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,...,1500.0,NaN,NaN,NaN,2000.0,100.0,800.0,NaN,NaN,NaN
H0011001,2015-16,Central,NaN,NaN,NaN,NaN,NaN,8800.0,NaN,NaN,NaN,NaN,...,NaN,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H0020801,2015-16,Central,NaN,NaN,4000.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,...,3000.0,NaN,NaN,NaN,NaN,100.0,1000.0,NaN,NaN,NaN


Prices

In [103]:
#Import csv file
ugd_p0 = pd.read_csv('Data Files/Uganda - Food Prices (2015-16).csv')
#Create pivot table with correct indexing
ugd_p = pd.pivot_table(ugd_p0, values="price", index=["j","u"], columns=["t","m"] , aggfunc="median")
#Display
ugd_p.head()

t                     2015-16                                       
m                     Central      Eastern     Northern      Western
j             u                                                     
Apples        Kg  2444.678155  2765.445752  2817.179700  2280.574043
Avocado       Kg   772.047798   527.867923   577.427162   476.555939
Beans (dry)   Kg  2430.222603  2497.324159  5151.075269  2498.998822
Beans (fresh) Kg  3081.552288  2829.373691  4229.389140  1632.899600
Beef          Kg  8741.190476  7923.907563  8106.250000  8413.978491

Consumption

In [104]:
# Convert expenditures to long format
ugd_x_long = ugd_x.stack().reset_index(name="Expenditure")  # Ensure values are in a column
# Obtain prices in long (original) format
ugd_p_long = ugd_p0
# Merge expenditures with prices
ugd_c = ugd_x_long.merge(ugd_p_long, on=["j", "t", "m"], how="left")
# Compute consumption
ugd_c["Consumption"] = ugd_c["Expenditure"] / ugd_c["price"]
# Pivot back to the desired format
ugd_c = ugd_c.pivot_table(values="Consumption", index=["i","t","m","u"], columns="j", aggfunc="sum")
# Display
ugd_c.head()

,,,j,Apples,Avocado,Beans (dry),Beans (fresh),Beef,Beer,Bread,Cabbage (green),Carrots,Cassava (dry),...,Sugar,Sweet Bananas,Sweet Potatoes (dry),Sweet Potatoes (flour),Sweet Potatoes (fresh),Tea,Tomatoes,Water,Watermelon,Wheat (flour)
i,t,m,u,,,,,,,,,,,,,,,,,,,,,
H0010101,2015-16,Central,Kg,NaN,0.647628,1.234455,NaN,1.029608,NaN,0.546974,1.385410,NaN,NaN,...,1.789058,NaN,NaN,NaN,1.665972,0.019011,3.762935,NaN,NaN,NaN
H0010201,2015-16,Central,Kg,NaN,NaN,0.822970,NaN,NaN,NaN,0.218790,0.692705,NaN,NaN,...,0.662614,NaN,NaN,NaN,2.498958,0.014258,0.806343,NaN,NaN,NaN
H0010401,2015-16,Central,Kg,NaN,NaN,0.740673,NaN,NaN,0.779162,NaN,NaN,NaN,NaN,...,0.496960,NaN,NaN,NaN,1.665972,0.009505,1.075124,NaN,NaN,NaN
H0011001,2015-16,Central,Kg,NaN,NaN,NaN,NaN,NaN,1.714156,NaN,NaN,NaN,NaN,...,NaN,1.199924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H0020801,2015-16,Central,Kg,NaN,NaN,1.645940,NaN,1.144009,NaN,NaN,NaN,NaN,NaN,...,0.993921,NaN,NaN,NaN,NaN,0.009505,1.343905,NaN,NaN,NaN


HH Characteristics

In [17]:
#Import csv file
ugd_z = pd.read_csv('Data Files/Uganda - Household Characteristics.csv')
#Filter for just 2015-16 data
ugd_z = ugd_z[ugd_z['t'] == '2015-16']
#Set the index
ugd_z = ugd_z.set_index(['i','t','m'])
#Set column group to 'k'
ugd_z.columns.name = 'k'
#Fill null spaces with 0
ugd_z.fillna(0, inplace=True)
#Display
ugd_z.head()

,,k,F 00-03,M 00-03,F 04-08,M 04-08,F 09-13,M 09-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+,log HSize
i,t,m,,,,,,,,,,,,,,,
H0010101,2015-16,Central,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.098612
H0010201,2015-16,Central,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.098612
H0010401,2015-16,Central,1.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.945910
H0011001,2015-16,Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.693147
H0020801,2015-16,Central,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.386294


FCT

In [18]:
#Import csv file
ugd_fct = pd.read_csv('Data Files/Uganda - FCT.csv')
#Rename 'index' column as 'j' for goods
ugd_fct = ugd_fct.rename(columns={"index":"j"})
#Set column group to 'n'
ugd_fct.columns.name = 'n'
#Set index and display
ugd_fct.set_index(["j"]).head()

n,Energy,Protein,Fiber,Folate,Calcium,Carbohydrate,Iron,Niacin,Riboflavin,Thiamin,Vitamin A,Vitamin B-12,Vitamin B-6,Vitamin C,Zinc,Magnesium,Phosphorus,Potassium,Vitamin E,Vitamin K
j,,,,,,,,,,,,,,,,,,,,
Apples,617.893,1.875,20.43,0,46.56,147.817,0.0,0.90,0.6625,0.0875,0,0.0,0.2131,0,0.1964,46.95,91.83,953.1,0.0,0
Avocado,1600.000,20.000,70.00,810,120.00,85.000,6.0,17.38,1.3000,0.6700,70,0.0,2.5700,100,6.0000,0.00,0.00,0.0,0.0,0
Beans (dry),1700.000,98.000,60.00,500,580.00,325.000,30.0,7.00,1.2000,3.4000,0,0.0,2.1500,10,8.0000,0.00,0.00,0.0,0.0,0
Beans (fresh),3470.000,214.000,160.00,5250,1130.00,626.000,51.0,11.74,2.1200,7.1300,0,0.0,4.7400,63,23.0000,0.00,0.00,0.0,0.0,0
Beef,2510.000,182.000,0.00,60,70.00,0.000,19.0,31.50,1.6000,0.9000,0,28.9,3.8000,0,37.0000,0.00,0.00,0.0,0.0,0


RDI

In [20]:
#Import csv file
ugd_rdi = pd.read_csv('Data Files/Uganda - RDA.csv')
#Set index 
ugd_rdi = ugd_rdi.set_index(["n"])
#Set column group to 'k'
ugd_rdi.columns.name = 'k'
#Display
ugd_rdi.head()

k,F 00-03,M 00-03,F 04-08,M 04-08,F 09-13,M 09-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
n,,,,,,,,,,,,,,
Energy,1000.0,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
Fiber,14.0,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
Folate,150.0,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
Calcium,700.0,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


### **USA Data**

Expenditures <br> 
*(Must run Consumption code first)*

In [94]:
# Merge food consumption with prices on 'food_code'
usa_x = usa_c0.merge(usa_p0, on=["food_code"], how="left").rename(columns={"t_x":"t"})
# Calculate expenditure (consumption * price per gram)
usa_x["expenditure"] = usa_x["u"] * usa_x["price"]
#Create pivot table
usa_x = pd.pivot_table(usa_x, values="expenditure", index=["i","t","m"], columns="j", aggfunc="median", fill_value=0)
# Display
usa_x.head()

,,j,100 grand bar,3 musketeers bar,Agave liquid sweetener,"Alcoholic malt beverage, sweetened","Alfalfa sprouts, raw",Alfredo sauce,Almond butter,"Almond milk, sweetened","Almond milk, sweetened, chocolate","Almond milk, unsweetened",...,"Yogurt, nonfat milk, fruit","Yogurt, nonfat milk, plain","Yogurt, ns as to type of milk, flavors other than fruit","Yogurt, ns as to type of milk, fruit","Yogurt, ns as to type of milk, plain","Yogurt, soy","Yogurt, whole milk, baby food","Yogurt, whole milk, flavors other than fruit","Yogurt, whole milk, fruit","Yogurt, whole milk, plain"
i,t,m,,,,,,,,,,,,,,,,,,,,,
83732,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83733,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83734,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83735,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83736,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Prices <br>
*(Note: no distinction on `m`, as the only market is 'USA')*

In [95]:
usa_p0 = pd.read_csv('Data Files/USA - Food Prices.csv')
#Filter for year and remove columns we aren't using
usa_p0 = usa_p0[usa_p0["year"] == "2015/2016"].drop(labels=['mod_code','method_description','method','nhanes'], axis=1)
#Rename columns using food name instead of food code
usa_p0 = usa_p0.rename(columns={'year':'t'})
code2namedf = usa_x0.drop(labels=['i','t','m','u'], axis=1).drop_duplicates()
code2name = code2namedf.set_index(code2namedf.columns[0])[code2namedf.columns[1]].to_dict()
usa_p1 = usa_p0.copy()
usa_p1 = usa_p1.rename(columns={'food_code':'j'})
usa_p1["j"] = usa_p1["j"].map(code2name)
#Create pivot table
usa_p = pd.pivot_table(usa_p1, values="price", index="j", columns="t" , aggfunc="median", fill_value=0)
#Display
usa_p.head()

t,2015/2016
j,
100 grand bar,0.941804
3 musketeers bar,1.225896
Agave liquid sweetener,0.616332
"Alcoholic malt beverage, sweetened",0.372661
"Alfalfa sprouts, raw",1.532246


Consumption

In [96]:
#Import csv file
usa_c0 = pd.read_csv('Data Files/USA - Food Expenditures 2015.csv')
#Rename columns
usa_c0 = usa_c0.rename(columns={'j':'food_code','food_name':'j','grams':'u'})
#Drop data where food price isn't available
usa_c0 = usa_c0[usa_c0["food_code"].isin(usa_p0["food_code"])]
#Create pivot table with correct indexing
usa_c = pd.pivot_table(usa_c0, values="u", index=["i","t","m"], columns="j" , aggfunc="median", fill_value=0)
#Display
usa_c.head()

,,j,100 grand bar,3 musketeers bar,Agave liquid sweetener,"Alcoholic malt beverage, sweetened","Alfalfa sprouts, raw",Alfredo sauce,Almond butter,"Almond milk, sweetened","Almond milk, sweetened, chocolate","Almond milk, unsweetened",...,"Yogurt, nonfat milk, fruit","Yogurt, nonfat milk, plain","Yogurt, ns as to type of milk, flavors other than fruit","Yogurt, ns as to type of milk, fruit","Yogurt, ns as to type of milk, plain","Yogurt, soy","Yogurt, whole milk, baby food","Yogurt, whole milk, flavors other than fruit","Yogurt, whole milk, fruit","Yogurt, whole milk, plain"
i,t,m,,,,,,,,,,,,,,,,,,,,,
83732,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83733,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83734,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83735,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83736,2015,USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


HH Characteristics

In [101]:
#Import csv file
usa_z = pd.read_csv('Data Files/USA - Household Characteristics 2015.csv')
#Filter for just 2015 data
usa_z = usa_z[usa_z['t'] == 2015]
#Set the index
#usa_z = usa_z.set_index(['i','t','m'])
#Set column group
#usa_z.columns.name = 'k'
#Fill null spaces with 0
#usa_z.fillna(0, inplace=True)
#Display
usa_z.head()

,i,t,m,age,sex,HH_size
0,83732,2015,USA,62.0,Male,2
1,83733,2015,USA,53.0,Male,1
2,83734,2015,USA,78.0,Male,2
3,83735,2015,USA,56.0,Female,1
4,83736,2015,USA,42.0,Female,5


FCT

In [99]:
#Import nutrient data
fct_unfiltered = pd.read_csv('Data Files/USA - FCT.csv')
#Filter for matches in food expenditures df
usa_fct = fct_unfiltered[fct_unfiltered["Ingredient description"].isin(usa_c0["j"])]
usa_fct.head()

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60
5,1006,"Cheese, brie",0.673,0.504,3.065,8.246,1.007,2.880,6.563,0.513,...,1.65,0.0,0.235,0.0,0.5,0.24,0.0,2.3,48.42,2.38
7,1009,"Cheese, cheddar",0.820,0.935,3.005,8.625,0.504,3.394,7.403,1.171,...,1.10,0.0,0.066,0.0,0.6,0.71,0.0,2.4,37.02,3.64
8,1011,"Cheese, colby",0.597,0.422,3.831,8.232,0.979,3.630,7.813,0.676,...,0.83,0.0,0.079,0.0,0.6,0.28,0.0,2.7,38.20,3.07
9,1012,"Cheese, cottage, creamed, large or small curd",0.064,0.070,0.263,0.778,0.047,0.345,0.714,0.105,...,0.43,0.0,0.046,0.0,0.1,0.08,0.0,0.0,79.79,0.40


RDI

In [100]:
usa_rdi = pd.read_csv('Data Files/USA - RDA.csv')
usa_rdi = usa_rdi.drop(labels="Constraint Type", axis=1).set_index(["Nutrient"])
usa_rdi.head()

,Nutrient Type,Unit,Child_1_3,Female_4_8,Male_4_8,Female_9_13,Male_9_13,Female_14_18,Male_14_18,Female_19_30,Male_19_30,Female_31_50,Male_31_50,Female_51U,Male_51U
Nutrient,,,,,,,,,,,,,,,
Energy,Macro,kcal,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,Macro,g,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
Carbohydrate,Macro,g,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
Dietary Fiber,Macro,g,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
Linoleic Acid,Macro,g,7.0,10.0,10.0,10.0,12.0,11.0,16.0,12.0,17.0,12.0,17.0,11.0,14.0
